# Notebook Edge detection (lectures 4 & 5)

*Notes explicatives et pratiques des notions abordées.*   

## Linear Systems
*interprétation personnelle*\
Dans le cadre de l'utilisation de filtres, on peut qualifié de système une fonction permettant de passer d'une représentation fonctionnelle de l'image à une autre. En effet, si l'on se réfère au chapitre précédant, nous pouvons imaginer une image comme une fonction discrète 2D (définie pour des valeurs entières représentant les indices de ligne et de colonne au sein de l'image).
Nous pouvons flouter l'image, dans ce cas là l'image correspond à une autre fonction discrète.

La transformation (et même plus précisément le filtre) permettant de passer de la première représentation de l'image (la première fonction) à la seconde représentation (la seconde fonction) est qualifié de système. 

Si le système avec lequel nous travaillons est qualifié de linéaire, alors si nous appliquons 2 transformations appartenant au système, sur notre image, alors nous devons avoir:
$$
S[\alpha f_i[n, m] + \beta f_j[n, m]] = \alpha S[f_i[n, m]] + \beta S[f_j[n, m]]
$$

Cela signifie que la résultante des 2 transformations est identiques à somme (pondérées) de chacune des résultantes prises séparément.

In [ ]:
import numpy as np
import cv2
from scipy.signal import convolve2d

origin_arr = np.identity(10) + np.rot90(np.identity(10))

In [ ]:
output1 = origin_arr / np.sum(origin_arr)
output2 =  2 * origin_arr - 2

In [ ]:
0.5 * output1 + 0.5 * output2

## 3. Edge Detection for Computer Vision
Le but de la détection des bords est d'identifier les variations soudaines (les discontinuités) au sein d'une image.
De manière intuitive, la majorité de l'information sémantique peut être associée aux bords.

Les bords nous aident à extraire des informations dans ce que nous voyons, à reconnaître des objets, des formes géométriques et appréhender le point de vue avec lequel nous observons une scène.
Les bords se "forment"/définissent par la discontinuité d'une surface, la variation brutale de la profondeur, des couleurs ou de la luminosité.

### 3.1 Les dérivées discrètres en 1D:
> **Définition 1:** La dérivée *"vers l'arrière"*
>
> $$
  \frac{df}{dx} = f(x) - f(x-1) = f'(x)
  $$

> **Définition 2:** La dérivée *"vers l'avant"*
>
> $$
  \frac{df}{dx} = f(x) - f(x+1) = f'(x)
  $$


> **Définition 3:** La dérivée *"centrée"*
>
> $$
  \frac{df}{dx} = f(x+1) - f(x-1) = f'(x)
  $$


In [ ]:
def apply_derivative(arr:np.array, derivative: str):
    """ Depending of the derivative function, the first, last or first and last elements of the array must be skipped

    Args:
        arr (np.array): 1D array to differenciate
        derivative (str): name of the definition of the derivative applied ("backward", "forward" or "centred")
    """
    output = []
    if derivative == "backward":
        output.append(np.nan) # None pour signifier que l'on skip en theorie la premiere dérivée
        for i in range(1, len(arr)):
            diff = arr[...] - arr[...]
            output.append(diff)
    elif derivative == "forward":
        for i in range(len(arr) - 1):
            diff = arr[...] - arr[...]
            output.append(diff)
        output.append(np.nan) # None pour signifier que l'on skip en theorie la premiere dérivée
    elif derivative == "centred":
        output.append(np.nan) # None pour signifier que l'on skip en theorie la premiere dérivée
        for i in range(1, len(arr) - 1):
            diff = arr[...] - arr[...]
            output.append(diff)
        output.append(np.nan)  # None pour signifier que l'on skip en theorie la derniere dérivée
    return np.array(output)

In [ ]:
arr = np.random.randint(0, 10, size=15)

diff_backward = apply_derivative(arr, "backward")
diff_forward = apply_derivative(arr, "forward")
diff_centred = apply_derivative(arr, "centred")

In [ ]:
print("input array  :", arr.reshape(1, -1))
print("backward diff:", diff_backward.reshape(1, -1))
print("forwward diff:", diff_forward.reshape(1, -1))
print("centred diff :", diff_centred.reshape(1, -1))

Voici un exemple de ce que j'ai pu obtenir:
```python
input array  : [[3 3 0 9 7 2 7 0 2 1 1 8 3 0 4]]
backward diff: [[None 0 -3 9 -2 -5 5 -7 2 -1 0 7 -5 -3 4]]
forwward diff: [[0 3 -9 2 5 -5 7 -2 1 0 -7 5 3 -4 None]]
centred diff : [[None -3 6 7 -7 0 -2 -5 1 -1 7 2 -8 1 None]]
```

### 3.2 Discrete derivative in 2D:
> **Définition: Vecteur gradient**
> $$
  \nabla f(x, y) = \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix} 
  $$

> **Définition: Amplitude du vecteur gradient**
> $$
  \left\lVert \nabla f(x, y) \right\rVert = \sqrt{\frac{\partial f}{\partial x}^2 + \frac{\partial f}{\partial y}^2}
  $$

> **Définition: Direction du vecteur gradient**
> $$
  \theta = arctan\left(\frac{\frac{\partial f}{\partial y}}{\frac{\partial f}{\partial x}}\right)
  $$

### 3.3 Example (and approximation)
Le gradient d'une matrice aux coordonnées $(i,j)$ peut être approximé en utilisant un filtre faisant intervenir les valeurs des pixels autour du pixel d'intérêt (le pixel central de coordonnée $(i,j)$) via le filtre $\mathcal{F}_1$:
$$
\mathcal{F}_X
=
\frac{1}{3}
\begin{bmatrix}
-1 & 0 & 1 \\
-1 & 0 & 1 \\
-1 & 0 & 1
\end{bmatrix}
$$
Dans ce cas si, nous obtenons une approximation de la composante $\frac{\partial f}{\partial x}$.

Avec un second filtre $\mathcal{F}_Y$, nous pouvons obtenir une approximation de la seconde composant le long de l'axe $y$:
$$
\mathcal{F}_Y
=
\frac{1}{3}
\begin{bmatrix}
1 & 1 & 1 \\
0 & 0 & 0 \\
-1 & -1 & -1
\end{bmatrix}
$$

#### Exemple:
Si nous avons une image constituée d'une seule bande (niveau de gris) telle que:
$$
\mathcal{I}
=
\begin{bmatrix}
1 & 2 & 3 & 4 & 5 \\ 
11 & 12 & 13 & 14 & 15 \\ 
21 & 22 & 23 & 24 & 25 \\ 
31 & 32 & 33 & 34 & 35 \\ 
41 & 42 & 43 & 44 & 45 
\end{bmatrix}
$$

Comme pour la définition de la dérivée centrée, il n'est pas possible d'évaluer les valeurs de $\frac{\partial f}{\partial x}$ et $\frac{\partial f}{\partial y}$ sur les contours de l'image:
$$
\mathcal{F}_X \ast \mathcal{I}
=
\begin{bmatrix}
\texttt{undef} &  & \dots &  & \texttt{undef} \\
 & f_X\mathcal{I}_{2,1} & f_X \mathcal{I}_{2,2} & f_X \mathcal{I}_{2,3} &  \\ 
\vdots & f_X\mathcal{I}_{3,1} & f_X \mathcal{I}_{3,2} & f_X \mathcal{I}_{3,3} & \vdots \\ 
 & f_X\mathcal{I}_{4,1} & f_X \mathcal{I}_{4,2} & f_X \mathcal{I}_{4,3} &  \\ 
\texttt{undef} &  & \dots &  & \texttt{undef} \\
\end{bmatrix}
$$
avec $f_X \mathcal{I_{i,j}}$:
$$
f_X \mathcal{I_{i,j}}
=
\frac{1}{3}
\begin{bmatrix}
-1 & 0 & 1 \\
-1 & 0 & 1 \\
-1 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\mathcal{I}_{i-1, j-1} & \mathcal{I}_{i-1, j} & \mathcal{I}_{i-1, j+1} \\
\mathcal{I}_{i, j-1} & \mathcal{I}_{i, j} & \mathcal{I}_{i, j+1} \\
\mathcal{I}_{i+1, j-1} & \mathcal{I}_{i+1, j} & \mathcal{I}_{i+1, j+1} 
\end{bmatrix}
=
\frac{1}{3}((\mathcal{I}_{i-1, j+1} - \mathcal{I}_{i-1, j-1}) + (\mathcal{I}_{i, j+1} - \mathcal{I}_{i, j-1}) + (\mathcal{I}_{i+1, j+1} - \mathcal{I}_{i+1, j-1}))
$$

*Pour s'entraîner, vous pouvez dériver l'expression pour la 2eme composante du gradient.*

## 4 Simple Edge Detectors

### 4.1 Characterizing Edges
Un bord se définie comme une position où un changement rapide au sein de la fonction intensité d'une image.
En traçant la fonction intensité le long d'une ligne horizontale, les bords correspondent aux positions où la dérivée de la fonction intensité possède les valeurs les plus extrêmes (positives ou négatives).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
arr = np.array([[0, 0, 0, 5, 1, 1, 1, 1, 1, 1, 1],
                [0, 0, 0, 5, 1, 1, 1, 1, 1, 1, 1],
                [0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5],
                [0, 0, 0, 5, 1, 1, 1, 1, 1, 1, 1],
                [0, 0, 5, 1, 1, 1, 1, 1, 1, 1, 1]])



x = np.arange(11)
fig, axes = plt.subplots(5, 2, figsize=(12, 20))

axes[0][0].plot(x, ...) # tracer l'intensité (les valeurs) de la 1ere ligne de arr
axes[1][0].plot(x, ...) # tracer l'intensité (les valeurs) de la 2nd ligne de arr
axes[2][0].plot(x, ...) # tracer l'intensité (les valeurs) de la 3eme ligne de arr
axes[3][0].plot(x, ...) # tracer l'intensité (les valeurs) de la 4eme ligne de arr
axes[4][0].plot(x, ...) # tracer l'intensité (les valeurs) de la 5eme ligne de arr

axes[0][1].plot(x, ...) # tracer la différence d'intensité (1ere dérivée) de la 1ere ligne de arr
axes[1][1].plot(x, ...) # tracer la différence d'intensité (1ere dérivée) de la 2nd ligne de arr
axes[2][1].plot(x, ...) # tracer la différence d'intensité (1ere dérivée) de la 3eme ligne de arr
axes[3][1].plot(x, ...) # tracer la différence d'intensité (1ere dérivée) de la 4eme ligne de arr
axes[4][1].plot(x, ...) # tracer la différence d'intensité (1ere dérivée) de la 5eme ligne de arr

axes[0][0].set_title("ligne #0 de arr")
axes[1][0].set_title("ligne #1 de arr")
axes[2][0].set_title("ligne #2 de arr")
axes[3][0].set_title("ligne #3 de arr")
axes[4][0].set_title("ligne #4 de arr")

axes[0][1].set_title("dérivée de la ligne #0 de arr")
axes[1][1].set_title("dérivée de la ligne #1 de arr")
axes[2][1].set_title("dérivée de la ligne #2 de arr")
axes[3][1].set_title("dérivée de la ligne #3 de arr")
axes[4][1].set_title("dérivée de la ligne #4 de arr")

axes[0][0].grid()
axes[1][0].grid()
axes[2][0].grid()
axes[3][0].grid()
axes[4][0].grid()
axes[0][1].grid()
axes[1][1].grid()
axes[2][1].grid()
axes[3][1].grid()
axes[4][1].grid()

plt.show()

### 4.2 Gradient d'une image
Considérons une image: toute simple semblable avec le numpy array précédent.

Le gradient en 2 dimensions est définie de la manière suivante:
$$
\nabla f = \begin{bmatrix} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \end{bmatrix}
$$

La définition peut actuellement être étendue à $N$ dimension:
$$
\nabla f = \begin{bmatrix} \frac{\partial f}{\partial x_1} & \dots & \frac{\partial f}{\partial x_i} & \dots & \frac{\partial f}{\partial x_N} \end{bmatrix}
$$

Pour revenir sur le gradient de l'image (donc 2 dimensions), nous avons 2 composantes à calculer.

In [ ]:
import numpy as np
arr = np.array([[1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 254, 254, 254, 254, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1]])

In [ ]:
# Visualisons l'image
fig, ax = plt.subplots(1, 1, figsize=(7,7))
ax.imshow(arr, cmap="gray")

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='r', linestyle='-', linewidth=1)
plt.show()

In [ ]:
# calcul de df / dx de l'image simple
lst_rows = []
for row in arr:
    lst_rows.append(apply_derivative(row, 'backward'))
derivative_x_img = np.array(lst_rows)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_x_img)
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
plt.title(r"Valeur de la composante $\frac{\partial f}{\partial x}$", fontsize=18)
plt.show()

Nous pouvons observer que:
* la $6^{e}$ colonne de pixels possèdent comme valeurs $253\ (254 - 1)$, correspondant à la différence entre les colonnes $6^{e}$ et $5^{e}$ (car nous avons utilisé pour la dérivée la définition *backward*)
* la $8^{e}$ colonne de pixels possèdent comme valeur $-214$ résultant de la différence entre la $8^{e}$ et la $7^{e}$ colonnes, sauf pour le pixel $(7, 8)$,

In [ ]:
# calcul de df / dy de l'image simple
imgGray_transpose = arr.T 
lst_rows = []
for row in imgGray_transpose:
    lst_rows.append(apply_derivative(row, 'backward'))
derivative_y_img = np.array(lst_rows).T

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_y_img)
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
plt.title(r"Valeur de la composante $\frac{\partial f}{\partial y}$", fontsize=18)
plt.show()

*à vous de raisonner pour expliquer pourquoi la portion de ligne 7 et 8 ont des valeurs différentes de 0*

Nous pouvons également déterminer l'image correspondant à la direction du gradient d'intensité de l'image en chaque point de celle-ci:

$$
\left\lVert \nabla f(x, y) \right\rVert = \sqrt{\frac{\partial f}{\partial x}^2 + \frac{\partial f}{\partial y}^2}
$$


In [ ]:
# On peut desormais calculer l'amplitude du gradient d'intensité de l'image:
amplitude_derivative = np.sqrt(... ** 2 + ... ** 2)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(amplitude_derivative)
fig.colorbar(graph, ax=ax, label='amplitude')

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
plt.title("Variation de l'amplitude du vecteur gradient d'intensité", fontsize=14)
plt.show()

Nous déterminons l'image correspondant à la direction du gradient d'intensité de l'image en chaque point de celle-ci grâce à la définition:
$$
\theta = arctan\left(\frac{\frac{\partial f}{\partial y}}{\frac{\partial f}{\partial x}}\right)
$$

Nous obtenons une matrice où chaque élément $\theta_{i,j}$ est la valeur de l'angle du vecteur gradient d'intensité aux coordonnées $(i;j)$:
$$
\mathbf{\theta}
=
\begin{bmatrix}
\theta_{1,1}&      &                 &     \dots       &                 &      & \theta_{1, w}\\
            &\ddots&                 &                 &                 &      &              \\
            &      & \theta_{i-1,j-1}& \theta_{i-1,j}  & \theta_{i,j+1}  &      & \vdots       \\
\vdots      &      & \theta_{i,j-1}  & \theta_{i,j}    & \theta_{i,j+1}  &      &              \\
            &      & \theta_{i+1,j-1}& \theta_{i+1,j-1}& \theta_{i+1,j-1}&      &              \\
            &      &                 &                 &                 &\ddots&              \\
\theta_{h,1}&      &                 &   \dots         &                 &      & \theta_{h,w} \\
\end{bmatrix}
$$

Comprenons bien ce que représente le vecteur gradient d'intensité d'une image. Chaque pixels d'une image en noir et blanc possède une valeur. La dérivée de l'intensité d'une image donne 2 "images" pour chacune des composantes de la dérivée ($\frac{\partial f}{\partial x}$ et $\frac{\partial f}{\partial y}$). 


Tout comme un vecteur au sein d'un espace 2D, le vecteur gradient d'intensité peut se caractérisé par son amplitude et son angle avec les axes du repère:

![img](relation_composantes_amplitude_angle.png)

Rappelons que ces composantes reflètent la différence entre l'intensité de 2 pixels adjacents, donc la variation d'intensité d'un pixel à l'autre suivant les axes $x$ et $y$.

À partir des composantes du vecteur, nous pouvons déterminer l'amplitude (la longueur du vecteur) et la direction (l'angle entre le vecteur et l'axe $x$).
 
Comme pour les 2 composantes nous pouvons donc calculer une image pour l'amplitude (ce que nous avons fait juste avant) et une image pour l'angle du vecteur (comme juste après).

In [ ]:
derivative_x_img[np.isnan(derivative_x_img)] = 0
derivative_y_img[np.isnan(derivative_y_img)] = 0
theta = np.arctan(... / (... + 1e-8)) * (180 / np.pi) + 180 * (derivative_x_img < 0)

fig, ax = plt.subplots(1, 2, figsize=(14,8))
graph = ax[0].imshow(theta)
fig.colorbar(graph, ax=ax[0], label='angle (degree)')
ax[1].imshow(arr, cmap='gray')

# Major ticks
ax[0].set_xticks(np.arange(0, 14, 1))
ax[0].set_yticks(np.arange(0, 14, 1))
ax[1].set_xticks(np.arange(0, 14, 1))
ax[1].set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax[0].set_xticklabels(np.arange(1, 15, 1))
ax[0].set_yticklabels(np.arange(1, 15, 1))
ax[1].set_xticklabels(np.arange(1, 15, 1))
ax[1].set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax[0].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[0].set_yticks(np.arange(-.5, 14, 1), minor=True)
ax[1].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[1].set_yticks(np.arange(-.5, 14, 1), minor=True)

# grid of the axes
ax[0].grid(which='minor', color='black', linestyle='-', linewidth=1)
ax[1].grid(which='minor', color='red', linestyle='-', linewidth=1)


plt.show()

In [ ]:
# Quel est l'impact du bruit ?
# Reprenons la même image en ajoutant du random (ici assez gros car compris entre -10 et 10)
noised_img = arr + 20 * (np.random.random(size=arr.shape) -0.5)

In [ ]:
# Visualisons l'image
fig, ax = plt.subplots(1, 1, figsize=(14,8))
plt.imshow(noised_img, cmap="gray")

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='red', linestyle='-', linewidth=1)

plt.show()

In [ ]:
# calcul de df / dx de l'image simple
lst_rows = []
for row in noised_img:
    lst_rows.append(apply_derivative(row, 'backward').tolist())
derivative_x_img = np.array(lst_rows)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_x_img, cmap='viridis')
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

plt.show()

In [ ]:
# calcul de df / dy de l'image simple
imgGray_transpose = noised_img.T 
lst_rows = []
for row in imgGray_transpose:
    lst_rows.append(apply_derivative(row, 'backward').tolist())
derivative_y_img = np.array(lst_rows).T

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_y_img)
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

plt.show()

In [ ]:
# On peut desormais calculer l'amplitude du gradient d'intensité de l'image:
amplitude_derivative = np.sqrt(derivative_x_img ** 2 + derivative_y_img ** 2)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(amplitude_derivative)
fig.colorbar(graph, ax=ax, label='amplitude')

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

plt.show()

In [ ]:
# Nous pouvons également déterminer l'image correspondant à la direction du gradient d'intensité de l'image en chaque point de celle-ci:
derivative_x_img[np.isnan(derivative_x_img)] = 0
derivative_y_img[np.isnan(derivative_y_img)] = 0 
angle_derivative = ... + 180 * ...

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(angle_derivative, cmap='coolwarm')
fig.colorbar(graph, ax=ax, label='angle (degree)')

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

plt.show()

Maintenant à vous de jouer avec l'image de Mooncake (`mooncake.png`) !

![mooncake](mooncake.png)

In [ ]:
# Reccuparation de l'image de Mooncake en noir et blanc:
from PIL import Image

img = Image.open('mooncake.png')
imgGray = np.array(img.convert('L'), dtype='float') / 255.0
plt.imshow(imgGray, cmap="gray")
plt.show()

In [ ]:
# calcul de df / dx de mooncake
lst_rows = []
for row in ...:
    lst_rows.append(...)
derivative_x_img = np.array(...)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_x_img, cmap='coolwarm')
fig.colorbar(graph, ax=ax)
plt.show()

In [ ]:
# calcul de df / dy de mooncake
imgGray_transpose = ....T 
lst_rows = []
for row in ...:
    lst_rows.append(...)
derivative_y_img = np.array(...).T

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_y_img, cmap='coolwarm')
fig.colorbar(graph, ax=ax)
plt.show()

In [ ]:
# On peut desormais calculer l'amplitude du gradient d'intensité de l'image:
amplitude_derivative = ...

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(amplitude_derivative, cmap='coolwarm')
fig.colorbar(graph, ax=ax, label='amplitude')
plt.show()

In [ ]:
# Nous pouvons également déterminer l'image correspondant à la direction du gradient d'intensité de l'image en chaque point de celle-ci:
derivative_x_img[np.isnan(derivative_x_img)] = 0
derivative_y_img[np.isnan(derivative_y_img)] = 0 
angle_derivative = ...

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(angle_derivative, cmap='coolwarm')
fig.colorbar(graph, ax=ax, label='angle (degree)')
plt.show()

Maintenant regardons ce qu'il se passe si l'image est bruitée.

In [ ]:
noised_img = imgGray + 0.1 * np.random.random(size=imgGray.shape)

In [ ]:
# calcul de df / dx de mooncake
lst_rows = []
for row in ...:
    lst_rows.append(...)
derivative_x_img = np.array(...)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_x_img, cmap='coolwarm')
fig.colorbar(graph, ax=ax)
plt.show()

In [ ]:
# calcul de df / dy de mooncake
noised_img_transpose = ...
lst_rows = []
for row in ...:
    lst_rows.append(...)
derivative_y_img = ...

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_y_img, cmap='coolwarm')
fig.colorbar(graph, ax=ax)
plt.show()

In [ ]:
# On peut desormais calculer l'amplitude du gradient d'intensité de l'image:
amplitude_derivative = ...

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(amplitude_derivative, cmap='coolwarm')
fig.colorbar(graph, ax=ax, label='amplitude')
plt.show()

*Que remarquez vous entre l'amplitude du vecteur gradient d'intensité de l'image bruitée et de l'image non bruitée ?*

In [ ]:
# Nous pouvons également déterminer l'image correspondant à la direction du gradient d'intensité de l'image en chaque point de celle-ci:
derivative_x_img[np.isnan(derivative_x_img)] = 0
derivative_y_img[np.isnan(derivative_y_img)] = 0 
angle_derivative = ... + 180 * ...
# le terme "180 * ..." permet de prendre en compte les cas où la composante x est négative et ajoute +180°
fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(angle_derivative, cmap='coolwarm')
fig.colorbar(graph, ax=ax, label='angle (degree)')
plt.show()

*Que remarquez vous entre l'angle du vecteur gradient d'intensité de l'image bruitée et de l'image non bruitée ?*

## 4.4 Filtre Gaussien

Le filtre Gaussien a pour but de réduire le bruit au sein d'une image.

### Cas du filtre à 1 dimension:

$$
G(x) = \frac{1}{\sqrt{2\pi\sigma}}\exp\left(-\frac{x^2}{2\sigma^2}\right)
$$

In [ ]:
# À quoi ressemble une distribution gaussienne ?
x = np.linspace(-5, 5, 100)
sigma = [0.5, 0.75, 1.0, 1.5]

def f_gaussian(x, sigma):
    return (1 / (2 * np.pi * sigma**2)) * np.exp(- (x**2)/(2 * sigma**2))

y_0 = f_gaussian(x, sigma[0])
y_1 = f_gaussian(x, sigma[1])
y_2 = f_gaussian(x, sigma[2])
y_3 = f_gaussian(x, sigma[3])

fig, ax = plt.subplots(1, 1, figsize=(10,8))

ax.plot(x, y_0, label=r'distribution gaussienne $\sigma=0.5$')
ax.plot(x, y_1, label=r'distribution gaussienne $\sigma=0.75$')
ax.plot(x, y_2, label=r'distribution gaussienne $\sigma=1.0$')
ax.plot(x, y_3, label=r'distribution gaussienne $\sigma=1.5$')

plt.grid()
plt.legend()
plt.show()
# Nous pouvons voir que la "hauteur" de la courbe diminue en augmentant l'écart-type.

#### Définition d'un filtre gaussien
Maintenant que nous savons à quoi ressemble la fonction de distribution de Gauss, nous pouvons définir le filtre / kenel Gaussien.

Dans le cas d'une dimension, le kernel est à 1 dimension. En fonction de la taille du filtre que l'on souhaite ($3$, $5$, ...), nous construisons celui ci en appelant la fonction gaussienne en lui donnant $x = [-1, 0, 1]$ si nous souhaitons une taille de 3, $x = [-2, -1, 0, 1, 2]$ si nous souhaitons une taille de filtre de 5, etc ...

In [ ]:
filter_1 = f_gaussian(x=np.array([-1, 0, 1]), sigma=0.5)
filter_2 = f_gaussian(x=np.array([-1, 0, 1]), sigma=0.75)
filter_3 = f_gaussian(x=np.array([-1, 0, 1]), sigma=1.0)
filter_4 = f_gaussian(x=np.array([-1, 0, 1]), sigma=1.5)

normalized_1 = np.sum(filter_1)
normalized_2 = np.sum(filter_2)
normalized_3 = np.sum(filter_3)
normalized_4 = np.sum(filter_4)

print("Valeur du filtre gaussien de taille 3, avec \u03C3=0.5:", filter_1)
print("Valeur du filtre gaussien de taille 3, avec \u03C3=0.75:", filter_2)
print("Valeur du filtre gaussien de taille 3, avec \u03C3=1.0:", filter_3)
print("Valeur du filtre gaussien de taille 3, avec \u03C3=1.5:", filter_4)

Nous allons pouvoir convoluer le filtre avec un array d'une dimension pour voir le résultat:

In [ ]:
x = np.linspace(-10, 10, 400)
signal = 1 * np.sin(x)

slightly_noised = signal + 0.1 * (np.random.uniform(size=signal.shape) -0.5)
noised = signal + 0.5 * (np.random.uniform(size=signal.shape) -0.5)
strongly_noised = signal + (np.random.uniform(size=signal.shape) - 0.5)

fig, axes = plt.subplots(4, 1, figsize=(20,8))

axes[0].plot(signal)
axes[1].plot(slightly_noised)
axes[2].plot(noised)
axes[3].plot(strongly_noised)

[axes[i].grid() for i in range(4)]

plt.show()

In [ ]:
# Application de filtres gaussien
slightly_noised_f1 = np.convolve(slightly_noised, filter_1, 'same')
slightly_noised_f2 = np.convolve(slightly_noised, filter_2, 'same')
slightly_noised_f3 = np.convolve(slightly_noised, filter_3, 'same')
slightly_noised_f4 = np.convolve(slightly_noised, filter_4, 'same')
noised_f1 = np.convolve(noised, filter_1, 'same')
noised_f2 = np.convolve(noised, filter_2, 'same')
noised_f3 = np.convolve(noised, filter_3, 'same')
noised_f4 = np.convolve(noised, filter_4, 'same')
strongly_noised_f1 = np.convolve(strongly_noised, filter_1, 'same')
strongly_noised_f2 = np.convolve(strongly_noised, filter_2, 'same')
strongly_noised_f3 = np.convolve(strongly_noised, filter_3, 'same')
strongly_noised_f4 = np.convolve(strongly_noised, filter_4, 'same')

fig, axes = plt.subplots(3, 1, figsize=(20,22))

axes[0].plot(slightly_noised, c='blue', label='slighlty noised')
axes[0].plot(slightly_noised_f1, c='red', label=r'slighlty noised * $k_1$')
axes[0].plot(slightly_noised_f2, c='green', label=r'slighlty noised * $k_2$')
axes[0].plot(slightly_noised_f3, c='violet', label=r'slighlty noised * $k_3$')
axes[0].plot(slightly_noised_f4, c='pink', label=r'slighlty noised * $k_4$')

axes[1].plot(noised, c='blue', label='noised')
axes[1].plot(noised_f1, c='red', label=r'noised * $k_1$')
axes[1].plot(noised_f2, c='green', label=r'noised * $k_2$')
axes[1].plot(noised_f3, c='violet', label=r'noised * $k_3$')
axes[1].plot(noised_f4, c='pink', label=r'noised * $k_4$')

axes[2].plot(strongly_noised, c='blue', label='strongly_noised')
axes[2].plot(strongly_noised_f1, c='red', label=r'strongly_noised * $k_1$')
axes[2].plot(strongly_noised_f2, c='green', label=r'strongly_noised * $k_2$')
axes[2].plot(strongly_noised_f3, c='violet', label=r'strongly_noised * $k_3$')
axes[2].plot(strongly_noised_f4, c='pink', label=r'strongly_noised * $k_4$')

[axes[i].grid() for i in range(3)]
[axes[i].legend() for i in range(3)]
plt.show()

In [ ]:
diff_0 = (1 / signal.shape[0]) * (signal - slightly_noised)**2
diff_1 = (1 / signal.shape[0]) * (signal - slightly_noised_f1)**2
diff_2 = (1 / signal.shape[0]) * (signal - slightly_noised_f2)**2
diff_3 = (1 / signal.shape[0]) * (signal - slightly_noised_f3)**2
diff_4 = (1 / signal.shape[0]) * (signal - slightly_noised_f4)**2

tot_0 = np.sum(diff_0)
tot_1 = np.sum(diff_1)
tot_2 = np.sum(diff_2)
tot_3 = np.sum(diff_3)
tot_4 = np.sum(diff_4)

print(f"valeur de tot_0: {tot_0}")
print(f"valeur de tot_1: {tot_1}")
print(f"valeur de tot_2: {tot_2}")
print(f"valeur de tot_3: {tot_3}")
print(f"valeur de tot_4: {tot_4}")

Nous pouvons constater que l'application du filtre gaussien avec une valeur croissante de $\sigma$ atténue le bruit mais entraîne également une atténuation de l'amplitude du signal.

In [ ]:
# Application des filtres de taille 3, 5, 7, 9
k_3 = f_gaussian(x=np.array([-1, 0, 1]), sigma=1.0)
k_5 = f_gaussian(x=np.array([-2, -1, 0, 1, 2]), sigma=1.0)
k_7 = f_gaussian(x=np.array([-3, -2, -1, 0, 1, 2, 3]), sigma=1.0)
k_9 = f_gaussian(x=np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]), sigma=1.0)

slightly_noised_3 = np.convolve(slightly_noised, k_3, 'same')
slightly_noised_5 = np.convolve(slightly_noised, k_5, 'same')
slightly_noised_7 = np.convolve(slightly_noised, k_7, 'same')
slightly_noised_9 = np.convolve(slightly_noised, k_9, 'same')
noised_3 = np.convolve(noised, k_3, 'same')
noised_5 = np.convolve(noised, k_5, 'same')
noised_7 = np.convolve(noised, k_7, 'same')
noised_9 = np.convolve(noised, k_9, 'same')
strongly_noised_3 = np.convolve(strongly_noised, k_3, 'same')
strongly_noised_5 = np.convolve(strongly_noised, k_5, 'same')
strongly_noised_7 = np.convolve(strongly_noised, k_7, 'same')
strongly_noised_9 = np.convolve(strongly_noised, k_9, 'same')

fig, axes = plt.subplots(3, 1, figsize=(20,22))

axes[0].plot(slightly_noised, c='blue', label='slighlty noised')
axes[0].plot(slightly_noised_3, c='red', label=r'slighlty noised * $k_3$')
axes[0].plot(slightly_noised_5, c='green', label=r'slighlty noised * $k_5$')
axes[0].plot(slightly_noised_7, c='violet', label=r'slighlty noised * $k_7$')
axes[0].plot(slightly_noised_9, c='pink', label=r'slighlty noised * $k_9$')

axes[1].plot(noised, c='blue', label='noised')
axes[1].plot(noised_3, c='red', label=r'noised * $k_3$')
axes[1].plot(noised_5, c='green', label=r'noised * $k_5$')
axes[1].plot(noised_7, c='violet', label=r'noised * $k_7$')
axes[1].plot(noised_9, c='pink', label=r'noised * $k_9$')

axes[2].plot(strongly_noised, c='blue', label='strongly_noised')
axes[2].plot(strongly_noised_3, c='red', label=r'strongly_noised * $k_3$')
axes[2].plot(strongly_noised_5, c='green', label=r'strongly_noised * $k_5$')
axes[2].plot(strongly_noised_7, c='violet', label=r'strongly_noised * $k_7$')
axes[2].plot(strongly_noised_9, c='pink', label=r'strongly_noised * $k_9$')

[axes[i].grid() for i in range(3)]
[axes[i].legend() for i in range(3)]
plt.show()

In [ ]:
import numpy as np
arr = np.array([[1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 254, 254, 254, 254, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1],
                [1, 1, 1, 1, 1, 254, 254, 40, 40, 40, 40, 1, 1, 1]])

# Quel est l'impact du bruit ?
# Reprenons la même image en ajoutant du random (ici assez gros car compris entre -10 et 10)
noised_img = arr + 20 * (np.random.random(size=arr.shape) -0.5)

# Visualisons l'image
fig, ax = plt.subplots(1, 1, figsize=(14,8))
plt.imshow(noised_img, cmap="gray")

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='red', linestyle='-', linewidth=1)

plt.show()

In [ ]:
def f_gaussian_2D(size, sigma):
    x, y = np.arange((-size + 1) / 2, (size + 1) / 2, 1), np.arange(((-size + 1) / 2), ((size + 1) / 2), 1)
    xx, yy = np.meshgrid(x,y)
    gauss = np.exp(-(xx**2 + yy**2)/(2 * sigma**2)) / (2 * np.pi * sigma)
    return gauss / np.sum(gauss)

In [ ]:
k = f_gaussian_2D(3, 1)

print("valeur du kernel:\n", k)
print("somme des composantes du kenerl: ", k.sum())

In [ ]:
cv_gaussian_k = cv2.getGaussianKernel(3,1)
cv2_gaussian_k = cv_gaussian_k * cv_gaussian_k.T
print("valeur du kernel via OpenCV:\n", cv2_gaussian_k)
print("somme des composantes du kenerl: ", cv2_gaussian_k.sum())

Le kernel que nous avons définie et celui obtenue avec Open CV sont identiques

In [ ]:
blur_img = convolve2d(noised_img, k, mode='same')
cv_blur_img = cv2.GaussianBlur(noised_img, ksize=(3,3), sigmaX=1.0)

In [ ]:
# Visualisons l'image
fig, ax = plt.subplots(1, 1, figsize=(14,8))
graph = plt.imshow(cv_blur_img - blur_img, cmap="viridis")
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='red', linestyle='-', linewidth=1)

plt.show()

Nous avons une différence dans l'application du kernel. Qui conduit à des différences aux niveaux du passage du kernel sur les bords. Cela vient de la différence dans la réalisation de la convolution. Si nous utilisons la méthode `cv2.filter2D` d'OpenCV à la place de la méthode `convolve2d` de scipy, nous obtenons:

In [ ]:
blur_img = blur_img = cv2.filter2D(src=noised_img, ddepth=-1, kernel=k)

In [ ]:
# Visualisons l'image
fig, ax = plt.subplots(1, 1, figsize=(14,8))
graph = plt.imshow(cv_blur_img - blur_img, cmap="viridis", vmin=0, vmax=1)
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='red', linestyle='-', linewidth=1)

plt.show()

In [ ]:
# Visualisons l'image
fig, ax = plt.subplots(1, 3, figsize=(18,10))
ax[0].imshow(noised_img, cmap="gray")
ax[1].imshow(blur_img, cmap="gray")
ax[2].imshow(cv_blur_img, cmap="gray")

# Major ticks
ax[0].set_xticks(np.arange(0, 14, 1))
ax[0].set_yticks(np.arange(0, 14, 1))
ax[1].set_xticks(np.arange(0, 14, 1))
ax[1].set_yticks(np.arange(0, 14, 1))
ax[2].set_xticks(np.arange(0, 14, 1))
ax[2].set_yticks(np.arange(0, 14, 1))


# Labels for major ticks
ax[0].set_xticklabels(np.arange(1, 15, 1))
ax[0].set_yticklabels(np.arange(1, 15, 1))
ax[1].set_xticklabels(np.arange(1, 15, 1))
ax[1].set_yticklabels(np.arange(1, 15, 1))
ax[2].set_xticklabels(np.arange(1, 15, 1))
ax[2].set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax[0].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[0].set_yticks(np.arange(-.5, 14, 1), minor=True)
ax[1].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[1].set_yticks(np.arange(-.5, 14, 1), minor=True)
ax[2].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[2].set_yticks(np.arange(-.5, 14, 1), minor=True)


ax[0].set_title("noised image")
ax[1].set_title("gaussian blured image")
ax[2].set_title("OpenCV gaussian blured image")
ax[0].grid(which='minor', color='red', linestyle='-', linewidth=0.5)
ax[1].grid(which='minor', color='red', linestyle='-', linewidth=0.5)
ax[2].grid(which='minor', color='red', linestyle='-', linewidth=0.5)

plt.show()

Nous pouvons voir que le filtre a bien atténué le bruit. Au détriment du *"tranchant"* et de la localisation des bords. Néanmoins le filtre gausssien conserve mieux le tranchant des bords que le filtre de "moyennage" (voir la vidéo de computerphile sur le [gausssian blur](https://www.youtube.com/watch?v=C_zFhWdM4ic))

In [ ]:
# calcul de df / dx de l'image simple
lst_rows = []
for row in blur_img:
    lst_rows.append(apply_derivative(row, 'backward').tolist())
derivative_x_img = np.array(lst_rows)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_x_img)
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
plt.title(r"Valeur de la composante $\frac{\partial f}{\partial x}$", fontsize=18)
plt.show()

In [ ]:
# calcul de df / dy de l'image simple
blur_img_transpose = blur_img.T 
lst_rows = []
for row in blur_img_transpose:
    lst_rows.append(apply_derivative(row, 'backward').tolist())
derivative_y_img = np.array(lst_rows).T

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(derivative_y_img)
fig.colorbar(graph, ax=ax)

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
plt.title(r"Valeur de la composante $\frac{\partial f}{\partial y}$", fontsize=18)
plt.show()

In [ ]:
# On peut desormais calculer l'amplitude du gradient d'intensité de l'image:
amplitude_derivative = np.sqrt(derivative_x_img ** 2 + derivative_y_img ** 2)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
graph = ax.imshow(amplitude_derivative)
fig.colorbar(graph, ax=ax, label='amplitude')

# Major ticks
ax.set_xticks(np.arange(0, 14, 1))
ax.set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1))
ax.set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True)
ax.set_yticks(np.arange(-.5, 14, 1), minor=True)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
plt.title("Variation de l'amplitude du vecteur gradient d'intensité", fontsize=14)
plt.show()

In [ ]:
derivative_x_img[np.isnan(derivative_x_img)] = 0
derivative_y_img[np.isnan(derivative_y_img)] = 0
theta = ... + 180 * ...

fig, ax = plt.subplots(1, 2, figsize=(14,8))
graph = ax[0].imshow(theta)
fig.colorbar(graph, ax=ax[0], label='angle (degree)')
ax[1].imshow(blur_img, cmap='gray')

# Major ticks
ax[0].set_xticks(np.arange(0, 14, 1))
ax[0].set_yticks(np.arange(0, 14, 1))
ax[1].set_xticks(np.arange(0, 14, 1))
ax[1].set_yticks(np.arange(0, 14, 1))

# Labels for major ticks
ax[0].set_xticklabels(np.arange(1, 15, 1))
ax[0].set_yticklabels(np.arange(1, 15, 1))
ax[1].set_xticklabels(np.arange(1, 15, 1))
ax[1].set_yticklabels(np.arange(1, 15, 1))

# Minor ticks
ax[0].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[0].set_yticks(np.arange(-.5, 14, 1), minor=True)
ax[1].set_xticks(np.arange(-.5, 14, 1), minor=True)
ax[1].set_yticks(np.arange(-.5, 14, 1), minor=True)

# grid of the axes
ax[0].grid(which='minor', color='black', linestyle='-', linewidth=1)
ax[1].grid(which='minor', color='red', linestyle='-', linewidth=1)


plt.show()

Nous pouvons également observer l'impact sur une image plus grande:

In [ ]:
# Reccuparation de l'image de Mooncake en noir et blanc:
img = Image.open('mooncake.png')
imgGray = np.array(img.convert('L'), dtype='float')
noised_img = imgGray + 20 * (np.random.random(size=imgGray.shape) - 5)

fig, axes = plt.subplots(1, 2, figsize=(15, 20))
axes[0].imshow(imgGray, cmap="gray")
axes[1].imshow(noised_img, cmap="gray")

axes[0].set_title("Original gray image")
axes[1].set_title("noised gray image")
plt.show()

In [ ]:
k3 = f_gaussian_2D(3, 1)
k5 = f_gaussian_2D(5, 2)
k7 = f_gaussian_2D(7, 3)

In [ ]:
blur_img_k3 = convolve2d(noised_img, k3, mode='same')
blur_img_k5 = convolve2d(noised_img, k5, mode='same')
blur_img_k7 = convolve2d(noised_img, k7, mode='same')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 20))
axes[0][0].imshow(noised_img, cmap="gray")
axes[0][1].imshow(blur_img_k3, cmap="gray")
axes[1][0].imshow(blur_img_k5, cmap="gray")
axes[1][1].imshow(blur_img_k7, cmap="gray")

axes[0][0].set_title("noised gray image")
axes[0][1].set_title("blured gray image with k_size = 3 and \u03C3=1")
axes[1][0].set_title("blured gray image with k5 and \u03C3=2")
axes[1][1].set_title("blured gray image with k7 and \u03C3=3")
plt.show()

In [ ]:
# Je vous laisse continuer l'analyse des 2 composantes du vecteur gradient d'intensité,
# amplitude et angle du vecteur sur les différentes images plus ou moins floutées de Mooncake 